In [13]:
import praw
from prawcore.exceptions import RequestException, ResponseException, OAuthException
import pandas as pd
import time
import os
from datetime import datetime, timedelta
import requests


reddit = praw.Reddit(client_id='aC3FI1WNM2F5uaSSJF2rEw', 
                     client_secret='y7BQGM0bRxCVyPEh4oQnioS2rW57ZQ', 
                     user_agent='gryphonholder')


In [14]:
subreddit = reddit.subreddit('ADHD')

In [15]:
count = 0
posts = []

Version 1

In [16]:
subreddit = reddit.subreddit('ADHD')
count = 0
posts = []
processed_ids = set()  # This will store the IDs of the processed posts

# Check if a count file exists and load the count from there
if os.path.exists('count.txt'):
    with open('count.txt', 'r') as f:
        count = int(f.read())

# Check if a posts file exists and load the posts from there
if os.path.exists('ADHDposts.csv'):
    df = pd.read_csv('ADHDposts.csv')
    posts = df.to_dict('records')

start_time = time.time()  # Initialize start_time

while count < 300000:  # Adjust the condition for the desired total count
    try:
        for submission in subreddit.new(limit=None):
            if submission.id in processed_ids:
                continue  # Skip this post if it has been processed before
            posts.append({
                "id": submission.id,
                "title": submission.title,
                "text": submission.selftext,
                "date": submission.created_utc,
                "upvotes": submission.score,
            })
            count += 1
            processed_ids.add(submission.id)  # Add the post ID to the set of processed IDs
            if count % 1000 == 0:
                end_time = time.time()  # End time after 1000 extractions
                print(f"Extraction count: {count}")
                rate = 1000 / (end_time - start_time)
                print(f"Extraction rate (last 1000): {rate:.2f} extractions/second")
                print(f"Extraction time (last 1000): {1000/rate:.2f} seconds taken")
                print(f"Title of the post: {submission.title}")
                start_time = time.time()  # Reset start_time for next 1000 extractions
    except (RequestException, OAuthException, ResponseException) as e:
        print(f"An error occurred: {e}")
        print("Sleeping for 60 seconds before trying again.")
        time.sleep(60)
    finally:
        # Save the count to a file
        with open('count.txt', 'w') as f:
            f.write(str(count))


Extraction count: 1000
Extraction rate (last 1000): 429.83 extractions/second
Extraction time (last 1000): 2.33 seconds taken
Title of the post: What is the weirdest way you trick your ADHD brain?


KeyboardInterrupt: 

Version 2

In [ ]:

subreddit = reddit.subreddit("ADHD")  # replace with your subreddit

posts = []
start_time = time.time()

for count, submission in enumerate(subreddit.new(limit=10000), 1):
    posts.append({
        "id": submission.id,
        "title": submission.title,
        "text": submission.selftext,
        "score": submission.score
    })

    if count % 500 == 0:
        end_time = time.time()
        rate = 500 / (end_time - start_time)
        print(f"Extraction count: {count}")
        print(f"Extraction rate (last 500): {rate:.2f} extractions/second")
        print(f"Extraction time (last 500): {500/rate:.2f} seconds taken")
        print(f"Title of the last post: {submission.title}")
        start_time = time.time()

df = pd.DataFrame(posts)



Version 3

In [ ]:
# Define the subreddit
subreddit = reddit.subreddit('ADHD')

# Define the start and end dates
start_date = datetime(2023, 6, 1)
end_date = datetime(2020, 6, 23)  # Reddit's launch date

# Initialize an empty DataFrame to store the posts
posts_df = pd.DataFrame()

# Initialize a counter for the number of posts
post_count = 0

# Loop over each 3-day interval in the date range
current_date = start_date
while current_date > end_date:
    # Define the query string
    query = f'timestamp:{int((current_date - timedelta(days=3)).timestamp())}..{int(current_date.timestamp())}'

    # Fetch the posts
    posts = subreddit.search(query, sort='new', limit=None, time_filter='all')

    # Initialize a counter for the number of posts in this interval
    interval_post_count = 0

    # Initialize a variable to store the start time
    start_time = time.time()

    # Loop over the posts and append them to the DataFrame
    for post in posts:
        posts_df = posts_df.append({
            'title': post.title,
            'score': post.score,
            'id': post.id,
            'body': post.selftext
        }, ignore_index=True)

        # Increment the counters
        post_count += 1
        interval_post_count += 1

        # Print the title of the first post of the interval
        if interval_post_count == 1:
            print(f"Scraping posts from {current_date - timedelta(days=3)} to {current_date}")
            print(f"Title of the first post: {post.title}")

        # Print the rate of scraping every 10 seconds
        if time.time() - start_time >= 10:
            print(f"Rate of scraping: {interval_post_count / (time.time() - start_time):.2f} posts/second")
            start_time = time.time()

    # Print the number of posts scraped from the previous interval
    print(f"Number of posts scraped from the previous interval: {interval_post_count}")

    # Decrement the current date
    current_date -= timedelta(days=3)

    # Sleep for 30 seconds to avoid hitting Reddit's rate limit
    time.sleep(30)


Number of posts scraped from the previous interval: 0
Number of posts scraped from the previous interval: 0
Number of posts scraped from the previous interval: 0


KeyboardInterrupt: 

Version 4 - using pushshift API

In [ ]:
# Define the subreddit
subreddit = 'ADHD'

# Define the start and end dates
start_date = datetime(2023, 6, 1)
end_date = datetime(2020, 6, 23)  # Reddit's launch date

# Initialize an empty DataFrame to store the posts
posts_df = pd.DataFrame()

# Initialize a counter for the number of posts
post_count = 0

# Loop over each 3-day interval in the date range
current_date = start_date
while current_date > end_date:
    # Define the query string
    after = int((current_date - timedelta(days=3)).timestamp())
    before = int(current_date.timestamp())

    # Fetch the posts
    url = f"https://api.pushshift.io/reddit/search/submission/?subreddit={subreddit}&after={after}&before={before}&sort=desc"
    response = requests.get(url)
    data = response.json()

    # Initialize a counter for the number of posts in this interval
    interval_post_count = 0

    # Initialize a variable to store the start time
    start_time = time.time()

    # Loop over the posts and append them to the DataFrame
    for post in data['data']:
        posts_df = posts_df.append({
            'title': post['title'],
            'score': post['score'],
            'id': post['id'],
            'body': post['selftext']
        }, ignore_index=True)

        # Increment the counters
        post_count += 1
        interval_post_count += 1

        # Print the title of the first post of the interval
        if interval_post_count == 1:
            print(f"Scraping posts from {current_date - timedelta(days=3)} to {current_date}")
            print(f"Title of the first post: {post['title']}")

        # Print the rate of scraping every 10 seconds
        if time.time() - start_time >= 10:
            print(f"Rate of scraping: {interval_post_count / (time.time() - start_time):.2f} posts/second")
            start_time = time.time()

    # Print the number of posts scraped from the previous interval
    print(f"Number of posts scraped from the previous interval: {interval_post_count}")

    # Decrement the current date
    current_date -= timedelta(days=3)

    # Sleep for 30 seconds to avoid hitting Reddit's rate limit
    time.sleep(30)



KeyError: 'data'

Althea's Version (working)

In [ ]:
import praw
import pandas as pd
reddit = praw.Reddit(client_id ='zWndV5hUJ8XZYshw1V9axw',
                     client_secret ='4ovj54M6J9s4yS6alG7c58jKOa3TQA',
                     user_agent ='my user agent')
subs = ['bipolar', 'schizophrenia']
submissions_types = ['hot', 'controversial', 'new', 'rising', 'top']
for x in subs:
    posts = []
    subreddit = reddit.subreddit(x)

    for submission_type in submissions_types:
        submission_generator = getattr(subreddit, submission_type)(limit=1000)
        posts.extend([[submission.id, submission.title, submission.selftext, submission.score] for submission in submission_generator])
        print(f'{submission_type.capitalize()} posts from {x} subreddit scraped')

    df = pd.DataFrame(posts, columns=['id', 'title', 'text', 'score'])
    df.to_csv(f'{x}.csv')

Saving posts

In [ ]:
# Save the posts to a CSV file
df = pd.DataFrame(posts)
df.to_csv('ADHD_10K_notcleaned.csv', index=False)

In [ ]:
df

In [ ]:
duplicates = df[df.duplicated('title', keep=False)]
duplicate_counts = duplicates['title'].value_counts()



print(duplicate_counts)

In [ ]:
df2